### Edgar Moises Hernandez-Gonzalez
#### 27-07-20-26-08-20
#### Clasificacion binaria de imagenes motoras en EEG con CNN-2D
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Escalogramas CWT (3D)
##### Entrada de 90x384

In [1]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd
import time
import pywt
from sklearn.metrics import confusion_matrix
from keras.models import load_model

In [5]:
c3_test = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/MIEEG_test.csv", header=None).iloc[:, 0:768]
c4_test = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/MIEEG_test.csv", header=None).iloc[:, 1536:2304]

In [6]:
x_test = pd.concat([c3_test, c4_test], axis=1)

In [7]:
y_test = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/etiquetas_test.csv", header=None)

In [8]:
print(x_test.shape)
print(y_test.shape)

(140, 1536)
(140, 1)


In [9]:
#unir 2 canales y 3 segundos
def unir_escalogramas_vertical(data):
  fs = 128
  escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,0.5)) / (1/fs)
  datos = np.zeros((data.shape[0],90,384))
  temporal = np.zeros((90,384))
  for i in range(data.shape[0]): #n muestras
    for j in range(2): #2 canales
      sig = data.iloc[i, j*768:(j+1)*768]
      sig = sig[32:416]
      # escalograma CWT
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3', sampling_period = (1 / fs))
      temporal[j*45:(j+1)*45, :] = abs(coef)
    datos[i] = temporal
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [10]:
#llamar a unir_escalogramas_vertical(data)
x_test = unir_escalogramas_vertical(x_test)

0
100


In [11]:
print(x_test.shape)

(140, 90, 384)


In [12]:
#convertir de 3D a 4D
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(140, 90, 384, 1)


In [13]:
model = load_model('/content/drive/My Drive/BCI-II-III/Modelos/CWT_CNN.hdf5')

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 90, 384, 8)        80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 45, 192, 8)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 45, 192, 8)        584       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 96, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16896)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                540704    
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)               

In [17]:
inicio = time.time()

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.8190627098083496


In [ ]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.4289
accuracy: 0.8143


In [ ]:
probabilidades = model.predict(x_test)
y_pred = np.argmax(probabilidades, 1) #1=fila

In [ ]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[62  8]
 [18 52]]


In [18]:
inicio = time.time()

probabilidades = model.predict(x_test[0].reshape(1,90,384,1))
y_pred = np.argmax(probabilidades, 1) #1=fila

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.05191516876220703
